In [1]:
import numpy as np
import pysb

In [22]:
class Volume:
    def __init__(self, sides, spacing, molecule_types, pad=True):
        # sides in nm
        # spacing in nm
        # pad adds one extra cell each direction
        if isinstance(sides, int):
            self.sidex = sides
            self.sidey = sides
        else:
            self.sidex, self.sidey = sides
        self.spacing = spacing
        self.molecule_types = molecule_types
        self.molecule_count = {mtype: 0 for mtype in self.molecule_types}
        self.pad = pad
    def partition(self):
        # creates an list of list of subvolumes
        pass
    def add_molecules(self, molecule_type, molecule_count):
        self.molecule_count[molecule_type] = molecule_count
        # distribute molecules randomly
        pass
    
class Subvolume:
    def __init__(self, uid, molecule_count={}):
        self.uid = uid
        self.molecule_count = molecule_count
    def add_molecule(self, molecule_type, molecule_count):
        self.molecule_count[molecule_type] += molecule_count
    
class Molecule:
    def __init__(self, molecule_type, diffusion_coefficient):
        self.molecule_type = molecule_type
        self.diffusion_coefficient = diffusion_coefficient

class ReactionSet:
    def __init__(self):
        self.reaction_set = {}
    def add_reaction(self, reagent, reaction_coefficient):
        # reagent can be [typeA, typeB] for bimolecular reaction
        # or [typeA] or typeA for unimolecular reaction
        self.reaction_set[frozenset(reagent)] = reaction_coefficient
        

In [23]:
a = Volume(sides=[8, 8], spacing=1, molecule_types=['A','B'], )

In [24]:
r = ReactionSet()

In [25]:
r.add_reaction(reagent=['A', 'B'], reaction_coefficient=3.5e-5)

In [26]:
r.reaction_set

{frozenset({'A', 'B'}): 3.5e-05}

In [29]:
r.reaction_set[frozenset(['A','B'])]

3.5e-05

2